**Team Members: Ethan Wong, Timmy Ren, Mason Shu, Medha Nalamada, Carson Mullen, Bethel Kim**

**Morning Cohort (11 AM - 1 PM)**

*Note to all: Please pull any changes from the repo before working on this file!*

In [ ]:
# Install Libraries
#!pip install deep-translator
#!pip install langdetect

In [3]:
# Import Libraries
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException,
    NoSuchElementException,
    ElementClickInterceptedException,
    WebDriverException,
)
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import numpy as np
import json
from deep_translator import GoogleTranslator
from langdetect import detect

# Task A: Scrape from ratebeer.com - extract 5-6K reviews

In [ ]:
def handle_cookie_consent():
    try:
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.ID, "onetrust-banner-sdk"))
        )
        accept_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@id='onetrust-accept-btn-handler']"))
        )
        accept_button.click()
        WebDriverWait(driver, 5).until(
            EC.invisibility_of_element_located((By.ID, "onetrust-banner-sdk"))
        )
    except (TimeoutException, NoSuchElementException, WebDriverException):
        pass

def handle_ratebeer_banner():
    try:
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'RateBeerBestBanner___StyledDiv-jAauHc'))
        )
        close_button = driver.find_element(By.CSS_SELECTOR, '.RateBeerBestBanner___StyledDiv-jAauHc .MuiIconButton-root')
        close_button.click()
        WebDriverWait(driver, 5).until(
            EC.invisibility_of_element_located((By.CLASS_NAME, 'RateBeerBestBanner___StyledDiv-jAauHc'))
        )
    except (TimeoutException, NoSuchElementException, WebDriverException):
        pass

json_file_path = 'beer_reviews.json'
with open(json_file_path, 'w', encoding='utf-8') as f:
    f.write('[')

chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()), options=chrome_options
)
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

# Navigate to the RateBeer Top Beers page
driver.get("https://www.ratebeer.com/top-beers")

handle_cookie_consent()

WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located(
        (By.XPATH, "//div[contains(@class,'DataTable__Row')]/div[2]/a")
    )
)

# Get the list of beers and their links
beer_elements = driver.find_elements(By.XPATH, "//div[contains(@class,'DataTable__Row')]/div[2]/a")
beers = []
for beer_element in beer_elements:
    full_text = beer_element.text
    product_name = full_text.split('\n')[0]  # Take the first line before the newline
    product_link = beer_element.get_attribute('href')
    beers.append({'name': product_name, 'link': product_link})

total_reviews_collected = 0
max_reviews = 30000  # Set your desired number of reviews to collect
first_review = True  # Flag to handle commas in JSON

# Loop through all beers
for i in range(len(beers)):
    product_name = beers[i]['name']
    product_link = beers[i]['link']

    # Navigate to the beer page
    driver.get(product_link)

    handle_cookie_consent()
    handle_ratebeer_banner()

    # Wait for the dropdown to be present
    try:
        dropdown = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "MuiSelect-selectMenu"))
        )
        dropdown.click()

        # Wait for the dropdown options to be visible
        option_100 = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, "//li[contains(text(), '100')]"))
        )
        option_100.click()
    except (TimeoutException, NoSuchElementException, WebDriverException):
        continue

    # Get the total number of reviews
    try:
        num_items_text = driver.find_element(By.CSS_SELECTOR, ".MuiTablePagination-selectRoot + .MuiTypography-colorInherit").text
        total_reviews = int(num_items_text.split(" ")[-1])
        num_pages = int(np.ceil(total_reviews / 100))
    except (NoSuchElementException, ValueError, WebDriverException):
        continue

    for j in range(num_pages):
        time.sleep(2)
        # Expand all "Show more" buttons
        while True:
            try:
                show_more_button = driver.find_element(By.XPATH, "//span[text()='Show more']")
                driver.execute_script("arguments[0].scrollIntoView(true);", show_more_button)
                show_more_button.click()
                time.sleep(1)
            except NoSuchElementException:
                break
            except ElementClickInterceptedException:
                handle_ratebeer_banner()
                driver.execute_script("arguments[0].click();", show_more_button)
            except Exception as e:
                print(f"Could not click 'Show more' button: {e}")
                break

        # Get all comments and store as a list
        comments_elements = driver.find_elements(By.CSS_SELECTOR, ".pre-wrap.MuiTypography-body1")
        product_reviews = [comment.text for comment in comments_elements]

        rating_elements = driver.find_elements(By.CSS_SELECTOR, ".bRPQdN.MuiTypography-subtitle1")
        ratings = [rating.text for rating in rating_elements]

        beer_data_list = []

        for review, rating in zip(product_reviews, ratings):
            beer_data = {
                'product_name': product_name,
                'product_review': review,
                'rating': rating
            }
            beer_data_list.append(beer_data)
            total_reviews_collected += 1

            if total_reviews_collected >= max_reviews:
                break

        # Write data to the JSON file
        with open(json_file_path, 'a', encoding='utf-8') as f:
            for beer_data in beer_data_list:
                if not first_review:
                    f.write(',\n')
                json.dump(beer_data, f, ensure_ascii=False)
                first_review = False
        beer_data_list = []

        if total_reviews_collected >= max_reviews:
            break

        if j + 1 < num_pages:
            try:
                button_to_click = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, ".MuiIconButton-colorInherit + .MuiIconButton-colorInherit"))
                )
                handle_ratebeer_banner()
                button_to_click.click()
                time.sleep(1)
            except ElementClickInterceptedException:
                handle_ratebeer_banner()
                driver.execute_script("arguments[0].click();", button_to_click)
            except Exception as e:
                print(f"Could not click 'Next' button: {e}")
                break

    if total_reviews_collected >= max_reviews:
        break

# Close the JSON array properly
with open(json_file_path, 'a', encoding='utf-8') as f:
    f.write(']')

driver.quit()

In [ ]:
# Load the JSON data into a DataFrame
with open('beer_reviews.json', 'r', encoding='utf-8') as file:
    beer_reviews = json.load(file)

df = pd.DataFrame(beer_reviews)
print(df['product_name'].value_counts())

In [ ]:
# Downsampling Reviews

In [ ]:
# Function to translate text to English if it's not already in English
def translate_to_english_if_needed(text):
    try:
        # Detect the language of the text
        lang = detect(text)
        # Translate only if the language is not English
        if lang != 'en':
            translated = GoogleTranslator(source='auto', target='en').translate(text)
            return translated
        else:
            return text
    except Exception as e:
        return str(e)

# Apply the translation function to the 'product_review' column
df['product_review'] = df['product_review'].apply(translate_to_english_if_needed)

print(df)

In [ ]:
# Export the DataFrame to a CSV or json file to prevent re-running the code in the future

# Task B: Develop a system where a customer specifies three attributes they desire in a beer, and the system recommends three beers that best match those attributes based on the extracted reviews

In [18]:
# Choose three attributes from the actual data
# Consider a word frequency analsysi of beer reviews to find important attributes in the actual data.
# Ensure attributes specified are likely to be mentioned together in a review (lift analysis) 

# Task C: Perform a similarity analysis using cosine similarity with the three attributes specified by the customer and the reviews

**Note: Used the bag-of-words model as opposed to word embeddings**

In [19]:
# Script should accept a file as input that has the product attributes and calculate similarity scores (0-1) 
    # between these attributes and each review
# Output fule should have three columns: product_name (will have a row for each review), product_review, 
    # and similarity_score

# Task D: Perform a sentiment analysis for every review

In [20]:
# Use VADER or any LLM

# Task E: Create an evaluation score for each beer that uses both similarity and sentiment scores and recommended three products to the customer

In [21]:
# Add sentiment and similarity scores for the three products recommended.

# Task F: How would our recommendations change if we use word vectors instead of the standard bag-of-words cosine similarity?

In [22]:
# Consider the % of reviews that mention a preferred attribute
    # For a recommended product, what % of its reviews mention an attribute specified by the customer
# Differences between bag-of-words and word vector approaches
# Bag of words, under certain conditions, will likely be better than word embeddings for recommender systems
# Show rating, similarity scores, and sentiments for the products recommended in this task and the previous one

# Task G: How would our recommendations differ if we ignored the similarity and feature sentiment scores and just chose the 3 highest rated products fro the entire data set? Would these products meet the requirements of the user looking for recommendations? Why or why not?

In [23]:
# Justify with analysis - use similarity and sentiment scores as well as overall ratings to answer this questions

# Task H: From 10 beers in the data, choose 1, and find the most similar beer among the remaining 9

In [24]:
# Show logic we are using in addition to finding the most similar product with markdown; explain method/logic